In [1]:
from path import Path
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

In [3]:
path = Path('../input')

### Datos de entrenamiento

In [4]:
train_df      = pd.read_csv(path/'train.csv')
train_weather = pd.read_csv(path/"weather_train.csv")

### Datos de test

In [5]:
test_df       = pd.read_csv(path/'test.csv')
test_weather  = pd.read_csv(path/"weather_test.csv")

### Edificios

In [6]:
building_df   = pd.read_csv(path/'building_metadata.csv')

### Opcional: Reducir memoria

In [7]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

train_df = reduce_mem_usage(train_df)
test_df  = reduce_mem_usage(test_df)

train_weather = reduce_mem_usage(train_weather)
test_weather  = reduce_mem_usage(test_weather)

building_df = reduce_mem_usage(building_df)

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)


### Metemos las columnas de meta-datos de edificios y climatología

In [8]:
train_df = train_df.merge(building_df, on='building_id', how='left')
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN


In [9]:
train_df = train_df.merge(train_weather, on=['site_id', 'timestamp'], how='left')
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0


### Qué atributos vamos a usar

In [10]:
features = ['square_feet', 'year_built', 'air_temperature', 'dew_temperature', 
            'sea_level_pressure', 'wind_direction', 'wind_speed', 'precip_depth_1_hr', 
            'floor_count']

### Definimos la x (entrada) y la y (salida o valor a predecir)

In [11]:
x = train_df[features].values
y = np.log1p(train_df[['meter_reading']].values.reshape(-1,1))

### Qué pasa con los NaNs?

In [12]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(x)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

In [13]:
x = imp.transform(x)

### Segmentación estratificada

In [29]:
import lightgbm as lgb
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

In [16]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'subsample': 0.25,
            'subsample_freq': 1,
            'learning_rate': 0.4,
            'num_leaves': 20,
            'feature_fraction': 0.9,
            'lambda_l1': 1,  
            'lambda_l2': 1
            }

In [30]:
folds = 4
seed = 42

kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

models = []
for fold,(train_idx, val_idx) in enumerate(tqdm(kf.split(train_df, train_df['building_id']),
                                               total=folds)):
    
    x_train,x_val = x[train_idx]            , x[val_idx]
    y_train,y_val = y[train_idx].squeeze(-1), y[val_idx].squeeze(-1)

    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_val   = lgb.Dataset(x_val,   y_val)
    
    gbm = lgb.train(
        params, lgb_train, num_boost_round=1000, valid_sets=(lgb_train, lgb_val),
        early_stopping_rounds=100, verbose_eval = 100)
    
    models.append(gbm)
    log_preds_val = gbm.predict(x_val)
    print(f'Fold: {fold} VAL RMSLE: {np.sqrt(mean_squared_error( y_val, log_preds_val))}')



  0%|          | 0/4 [00:00<?, ?it/s]

Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.69945	valid_1's rmse: 1.69899
[200]	training's rmse: 1.66531	valid_1's rmse: 1.66538
[300]	training's rmse: 1.64968	valid_1's rmse: 1.65027
[400]	training's rmse: 1.63895	valid_1's rmse: 1.64018
[500]	training's rmse: 1.63219	valid_1's rmse: 1.63416
[600]	training's rmse: 1.62762	valid_1's rmse: 1.63032
[700]	training's rmse: 1.62373	valid_1's rmse: 1.62708
[800]	training's rmse: 1.62088	valid_1's rmse: 1.62488
[900]	training's rmse: 1.61767	valid_1's rmse: 1.62233
[1000]	training's rmse: 1.61523	valid_1's rmse: 1.62032
Did not meet early stopping. Best iteration is:
[999]	training's rmse: 1.61523	valid_1's rmse: 1.62031




 25%|██▌       | 1/4 [07:38<22:55, 458.35s/it]

Fold: 0 VAL RMSLE: 1.620307247622671
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.70227	valid_1's rmse: 1.70346
[200]	training's rmse: 1.67146	valid_1's rmse: 1.67313
[300]	training's rmse: 1.6564	valid_1's rmse: 1.6586
[400]	training's rmse: 1.64601	valid_1's rmse: 1.64868
[500]	training's rmse: 1.63834	valid_1's rmse: 1.64166
[600]	training's rmse: 1.63301	valid_1's rmse: 1.63706
[700]	training's rmse: 1.62957	valid_1's rmse: 1.63434
[800]	training's rmse: 1.62553	valid_1's rmse: 1.63074
[900]	training's rmse: 1.62248	valid_1's rmse: 1.62825
[1000]	training's rmse: 1.62042	valid_1's rmse: 1.62675
Did not meet early stopping. Best iteration is:
[996]	training's rmse: 1.62042	valid_1's rmse: 1.62672




 50%|█████     | 2/4 [12:20<13:30, 405.40s/it]

Fold: 1 VAL RMSLE: 1.6267198691695068
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.702	valid_1's rmse: 1.70276
[200]	training's rmse: 1.66738	valid_1's rmse: 1.66872
[300]	training's rmse: 1.65192	valid_1's rmse: 1.65376
[400]	training's rmse: 1.64413	valid_1's rmse: 1.64663
[500]	training's rmse: 1.63607	valid_1's rmse: 1.63922
[600]	training's rmse: 1.63125	valid_1's rmse: 1.635
[700]	training's rmse: 1.62742	valid_1's rmse: 1.63182
[800]	training's rmse: 1.62462	valid_1's rmse: 1.62967
[900]	training's rmse: 1.62229	valid_1's rmse: 1.62791
[1000]	training's rmse: 1.61991	valid_1's rmse: 1.62605
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 1.61991	valid_1's rmse: 1.62605




 75%|███████▌  | 3/4 [17:03<06:08, 368.72s/it]

Fold: 2 VAL RMSLE: 1.6260458351398221
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 1.70015	valid_1's rmse: 1.69995
[200]	training's rmse: 1.66595	valid_1's rmse: 1.6662
[300]	training's rmse: 1.64902	valid_1's rmse: 1.64972
[400]	training's rmse: 1.63864	valid_1's rmse: 1.63997
[500]	training's rmse: 1.63336	valid_1's rmse: 1.63545
[600]	training's rmse: 1.62756	valid_1's rmse: 1.63025
[700]	training's rmse: 1.62378	valid_1's rmse: 1.62716
[800]	training's rmse: 1.62133	valid_1's rmse: 1.62527
[900]	training's rmse: 1.61944	valid_1's rmse: 1.62408
[1000]	training's rmse: 1.61727	valid_1's rmse: 1.62239
Did not meet early stopping. Best iteration is:
[999]	training's rmse: 1.61727	valid_1's rmse: 1.62238




100%|██████████| 4/4 [21:45<00:00, 342.63s/it]

Fold: 3 VAL RMSLE: 1.622384077022519


In [31]:
models

### Metemos las columnas de meta-datos de edificios y climatología (en test)

In [32]:
test_df = test_df.merge(building_df, on='building_id', how='left')
test_df = test_df.merge(test_weather, on=['site_id', 'timestamp'], how='left')
test_df.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01 00:00:00,0,Education,7432,2008.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
1,1,1,0,2017-01-01 00:00:00,0,Education,2720,2004.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
2,2,2,0,2017-01-01 00:00:00,0,Education,5376,1991.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
3,3,3,0,2017-01-01 00:00:00,0,Education,23685,2002.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
4,4,4,0,2017-01-01 00:00:00,0,Education,116607,1975.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609


In [33]:
test = test_df[features].values
test

array([[7.43200000e+03, 2.00800000e+03, 1.77968750e+01, ...,
        3.59960938e+00,            nan,            nan],
       [2.72000000e+03, 2.00400000e+03, 1.77968750e+01, ...,
        3.59960938e+00,            nan,            nan],
       [5.37600000e+03, 1.99100000e+03, 1.77968750e+01, ...,
        3.59960938e+00,            nan,            nan],
       ...,
       [1.12650000e+04, 1.99700000e+03,            nan, ...,
                   nan,            nan,            nan],
       [2.97750000e+04, 2.00100000e+03,            nan, ...,
                   nan,            nan,            nan],
       [9.22710000e+04, 2.00100000e+03,            nan, ...,
                   nan,            nan,            nan]])

In [34]:
test = imp.transform(test)
test

array([[7.43200000e+03, 2.00800000e+03, 1.77968750e+01, ...,
        3.59960938e+00, 7.96415539e-01, 4.18484813e+00],
       [2.72000000e+03, 2.00400000e+03, 1.77968750e+01, ...,
        3.59960938e+00, 7.96415539e-01, 4.18484813e+00],
       [5.37600000e+03, 1.99100000e+03, 1.77968750e+01, ...,
        3.59960938e+00, 7.96415539e-01, 4.18484813e+00],
       ...,
       [1.12650000e+04, 1.99700000e+03, 1.59879721e+01, ...,
        3.37750300e+00, 7.96415539e-01, 4.18484813e+00],
       [2.97750000e+04, 2.00100000e+03, 1.59879721e+01, ...,
        3.37750300e+00, 7.96415539e-01, 4.18484813e+00],
       [9.22710000e+04, 2.00100000e+03, 1.59879721e+01, ...,
        3.37750300e+00, 7.96415539e-01, 4.18484813e+00]])

### Predicciones

In [35]:
np.expm1(models[0].predict(test)), np.expm1(models[1].predict(test))

(array([  4.58055372,  12.67186137,   3.99810253, ...,   1.71519195,
        110.21820193,   2.87671095]),
 array([  4.5041803 ,   3.5712431 ,   1.52181083, ...,   0.91903019,
        135.89529701,   4.48414487]))

### Mezclamos (ensemble) los modelos (media)

In [36]:
models_preds = np.array([np.expm1(model.predict(test)) for model in models])
models_preds, models_preds.shape

(array([[  4.58055372,  12.67186137,   3.99810253, ...,   1.71519195,
         110.21820193,   2.87671095],
        [  4.5041803 ,   3.5712431 ,   1.52181083, ...,   0.91903019,
         135.89529701,   4.48414487],
        [  4.37739612,  11.06882549,   3.64885235, ...,   1.95063238,
         129.31316524,   5.54745404],
        [  2.83076547,   9.46972951,   1.0265634 , ...,   2.20772736,
         177.66973529,   4.82771251]]), (4, 41697600))

In [38]:
preds = np.mean(models_preds,axis=0)
preds

array([  4.0732239 ,   9.19541487,   2.54883227, ...,   1.69814547,
       138.27409987,   4.43400559])

### Preparemos el envío

In [39]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission

,row_id,meter_reading
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [40]:
sample_submission['meter_reading'] = preds
sample_submission.head()

,row_id,meter_reading
0,0,4.073224
1,1,9.195415
2,2,2.548832
3,3,9.677804
4,4,19.517782


### A disco en un CSV de toda la vida

In [41]:
csv_fname = '3-lgbm-kfold-estratificado.csv.zip'
sample_submission.to_csv(csv_fname, compression='zip', index=False)

### Opcional: enviar a Kaggle (OJO! que hay limite diario)

In [42]:
comp = 'ashrae-energy-prediction'
!kaggle competitions submit -c {comp} -f {csv_fname} -m 'lgbm con 4 folds (datos continuos)'

100%|████████████████████████████████████████| 319M/319M [00:09<00:00, 36.4MB/s]
Successfully submitted to ASHRAE - Great Energy Predictor III

In [43]:
# LB 1.69